In [34]:
import cv2
import os
import numpy as np

def detect_aruco_markers_and_save(image_path, save_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(image, (5, 5), 0)
    
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
    parameters = cv2.aruco.DetectorParameters_create()
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids, (0, 255, 0))
    height, width, _ = image.shape
    save_filename = os.path.join(save_path, 'detected_markers_image.jpg')
    cv2.imwrite(save_filename, image_with_markers)
    print(f"Image with detected markers saved at: {save_filename}")

    
    center_points = []
    for (marker, id) in zip(corners, ids):
        corner = marker.reshape((4, 2))
        (tl, tr, br, bl) = corner
        cx = int((tl[0] + br[0]) / 2)
        cy = int((tl[1] + br[1]) / 2)
        center_points.append((cx, cy))
        cv2.circle(image, (cx, cy), 4, (255, 1, 1), 2)  
        label_text = f'ID:{id[0]} ({cx},{cy})'
        # cv2.putText(image, label_text, (cx + 100, cy - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (1, 1, 1), 1, cv2.LINE_AA)

    center_points = np.array(center_points)
    hull = cv2.convexHull(center_points, clockwise=True, returnPoints=False)
    rect = cv2.minAreaRect(center_points)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    area = cv2.contourArea(box)
    

    (center, size, angle) = rect
    (width, height) = size
    
    
    print(f"Length of the rectangle: {width} pixels")
    print(f"Breadth of the rectangle: {height} pixels")

    mask = np.zeros_like(image)
    cv2.drawContours(mask, [box], 0, (255, 255, 255), -1)
    result_cropped = cv2.bitwise_and(image, mask)

    center = (int(rect[0][0]), int(rect[0][1]))
    radius = 50
    cv2.circle(result_cropped, center, radius, (0, 255, 0), 2)

    
    box_image = np.zeros_like(image)
    cv2.drawContours(box_image, [box], 0, (255,250, 0), -1)

    opacity = 0.25
    cv2.addWeighted(box_image, opacity, image, 1 - opacity, 0, image)
    

    
    # Save the cropped image
    save_filename_cropped = os.path.join(save_path, 'cropped_image.jpg')
    cv2.imwrite(save_filename_cropped, result_cropped)
    print(f"Cropped image inside the resultant box saved at: {save_filename_cropped}")
    save_filename_with_circles_and_box = os.path.join(save_path, 'image_with_circles_box.jpg')
    cv2.imwrite(save_filename_with_circles_and_box, image)
    print(f"Image with circles, blue box, and lines saved at: {save_filename_with_circles_and_box}")

image_path = '/home/qt/visionSort/visionSort_cv/Align.jpg'
save_path = '/home/qt/visionSort/visionSort_cv/imgs'
detect_aruco_markers_and_save(image_path, save_path)

Image with detected markers saved at: /home/qt/visionSort/visionSort_cv/imgs/detected_markers_image.jpg
Length of the rectangle: 1074.07861328125 pixels
Breadth of the rectangle: 774.8948364257812 pixels
Cropped image inside the resultant box saved at: /home/qt/visionSort/visionSort_cv/imgs/cropped_image.jpg
Image with circles, blue box, and lines saved at: /home/qt/visionSort/visionSort_cv/imgs/image_with_circles_box.jpg
